In [1]:
from google.colab import drive

drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd 

filepath_dict = {'SearchCreativeWork':'/content/drive/My Drive/Colab Notebooks/data/SearchCreativeWork.txt',
                 'GetWeather': '/content/drive/My Drive/Colab Notebooks/data/GetWeather.txt',
                 'BookRestaurant': '/content/drive/My Drive/Colab Notebooks/data/BookRestaurant.txt',
                 'PlayMusic': '/content/drive/My Drive/Colab Notebooks/data/PlayMusic.txt',
                 'AddToPlaylist': '/content/drive/My Drive/Colab Notebooks/data/AddToPlaylist.txt',
                 'RateBook': '/content/drive/My Drive/Colab Notebooks/data/RateBook.txt',
                 'SearchScreeningEvent':'/content/drive/My Drive/Colab Notebooks/data/SeachScreeningEvent.txt'
                 }

df_list = []
i = 0
for source,filepath in filepath_dict.items():
  df = pd.read_csv(filepath,names =['sentence'],sep='\t')
  df['label'] = i 
  df['source'] = source #cot them vao ten cua cau nhu ratebook,playmusic
  df_list.append(df)
  i = i + 1 

df = pd.concat(df_list)
print(df.head())


                                            sentence  label              source
0    find the soundtrack titled This Side of Para...      0  SearchCreativeWork
1                find a book called The Mad Magician      0  SearchCreativeWork
2                 find the picture Louder Than Bombs      0  SearchCreativeWork
3        show creative work of Through the Back Door      0  SearchCreativeWork
4  Where can I watch the television show You Make...      0  SearchCreativeWork


In [3]:
df.iloc[0]

sentence      find the soundtrack titled This Side of Para...
label                                                       0
source                                     SearchCreativeWork
Name: 0, dtype: object

In [4]:
df.tail()

,sentence,label,source
1955,Can a I get the movie schedule for Sympathy fo...,6,SearchScreeningEvent
1956,Find movie schedules for animated movies aroun...,6,SearchScreeningEvent
1957,what time is Bordertown Trail showing,6,SearchScreeningEvent
1958,in the neighbourhood find movies with movie times,6,SearchScreeningEvent
1959,What cinema has the closest movies,6,SearchScreeningEvent


In [0]:
# /content/drive/My Drive/Colab Notebooks/data

In [5]:
sentences = ['John likes ice cream','John hates chocolate.']
from sklearn.feature_extraction.text import CountVectorizer 
vectorizer = CountVectorizer(min_df = 0,lowercase = False)
vectorizer.fit(sentences)
vectorizer.vocabulary_

{'John': 0, 'chocolate': 1, 'cream': 2, 'hates': 3, 'ice': 4, 'likes': 5}

In [6]:
vectorizer.transform(sentences).toarray()

array([[1, 0, 1, 0, 1, 1],
       [1, 1, 0, 1, 0, 0]])

Ở đây chúng ta sẽ sử dụng lại trên mô hình BOW trước để vector hóa các câu. Có thể sử dụng lại CountVectorizercho nhiệm vụ này. Vì ta có thể không có sẵn dữ liệu kiểm tra trong quá trình đào tạo, bạn có thể tạo từ vựng chỉ bằng dữ liệu đào tạo. Sử dụng từ vựng này, ta có thể tạo các vectơ đặc trưng cho từng câu của tập huấn luyện và kiểm tra:

In [7]:
from sklearn.model_selection import train_test_split  
from keras.preprocessing.text import Tokenizer                    
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

for source in df['source'].unique():
    df_source = df['source']
    sentence = df_source.values
    y = df['label'].values

    sentences_train, sentences_test, y_train, y_test = train_test_split(
        sentence, y, test_size=0.25, random_state=1000)

    vectorizer = CountVectorizer()
    vectorizer.fit(sentences_train)
    X_train = vectorizer.transform(sentences_train)
    X_test  = vectorizer.transform(sentences_test)

    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    score = classifier.score(X_test, y_test)
    print('Accuracy for {} data: {:.4f}'.format(source, score))

Using TensorFlow backend.


Accuracy for SearchCreativeWork data: 1.0000
Accuracy for GetWeather data: 1.0000
Accuracy for BookRestaurant data: 1.0000
Accuracy for PlayMusic data: 1.0000
Accuracy for AddToPlaylist data: 1.0000
Accuracy for RateBook data: 1.0000
Accuracy for SearchScreeningEvent data: 1.0000


In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

input_dim = X_train.shape[1]

# # model = Sequential()
# # model.add(Dense(10, input_dim=input_dim, init='relu'))
# model = Sequential([
#     Dense(32, input_dim=input_dim),
#     Activation('relu'),
#     Dense(1),
#     Activation('softmax'),
# ])
# model.compile(loss='categorical_crossentropy',
#               optimizer=sgd,
#               metrics=['accuracy'])
from keras import layers

embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=input_dim, 
                           output_dim=embedding_dim, 
                           input_length= input_dim))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 50)             350       
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 871
Trainable params: 871
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy',
#               optimizer=sgd,
#               metrics=['accuracy'])


In [11]:
# And now?
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                80        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________


In [12]:
history = model.fit(X_train, y_train,
                    epochs=100,
                    validation_data=(X_test, y_test),
                    batch_size=10)
# model.fit(X_train, y_train,
#           nb_epoch=10,
#           batch_size=10)


Train on 10351 samples, validate on 3451 samples
Epoch 1/100
10351/10351 [==============================] - 5s 461us/step - loss: -21.8583 - accuracy: 0.1402 - val_loss: -67.9663 - val_accuracy: 0.1489
Epoch 2/100
10351/10351 [==============================] - 3s 301us/step - loss: -147.9412 - accuracy: 0.1437 - val_loss: -251.9807 - val_accuracy: 0.1489
Epoch 3/100
10351/10351 [==============================] - 3s 293us/step - loss: -371.2810 - accuracy: 0.1437 - val_loss: -522.4966 - val_accuracy: 0.1489
Epoch 4/100
10351/10351 [==============================] - 3s 296us/step - loss: -671.4361 - accuracy: 0.1437 - val_loss: -864.7435 - val_accuracy: 0.1489
Epoch 5/100
10351/10351 [==============================] - 3s 293us/step - loss: -1037.2983 - accuracy: 0.1437 - val_loss: -1271.2628 - val_accuracy: 0.1489
Epoch 6/100
10351/10351 [==============================] - 3s 296us/step - loss: -1465.2034 - accuracy: 0.1437 - val_loss: -1741.0791 - val_accuracy: 0.1489
Epoch 7/100
10351/1

In [0]:
score = model.evaluate(X_test, y_test, batch_size=10)

3451/3451 [==============================] - 1s 206us/step


In [0]:
loss, accuracy = model.evaluate(X_train,y_train,verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
>>> print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.1437


(-30.184682842936795, 0.1436576247215271)

In [0]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [0]:
plot_history(history)